# w261 Final Project - Clickthrough Rate Prediction
## Supplemental Notebook - Logistic Regression Pipelines

### Team 24   
### Vivian Lu, Siddhartha Jakkamreddy, Venky Nagapudi, Luca Garre   
### Summer 2019, sections 4 and 5   

## 4.3 Production Pipeline
### Logistic Regression Sample run and Full dataset 

### 4.3.1 Important articles to look at

https://towardsdatascience.com/apache-spark-mllib-tutorial-7aba8a1dce6e

https://stackoverflow.com/questions/32982425/encode-and-assemble-multiple-features-in-pyspark

https://blog.insightdatascience.com/spark-pipelines-elegant-yet-powerful-7be93afcdd42

https://medium.com/future-vision/spark-udfs-we-can-use-them-but-should-we-use-them-2c5a561fde6d

### 4.3.2 Basic pipeline based production model for Logistic Regression


In [ ]:
# read in  training data and convert to dataframe
start = time.time()
train_sample = sc.textFile('data/sample_train.txt')\
                 .map(lambda x: x.split('\t'))\
                 .toDF().repartition(16).write.parquet("sample_train")
print(f"\n... Completed in {time.time() - start} seconds")

In [299]:
#Read in dataframe - this is the tiny dataset with 100K lines
#Read in dataframe (need to do this in parquet)
start = time.time()
df = spark.read.parquet("sample_train")

df.show(2)
print(f"\n... Completed in {time.time() - start} seconds")

+---+---+---+---+---+----+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---+--------+--------+--------+--------+
| _1| _2| _3| _4| _5|  _6| _7| _8| _9|_10|_11|_12|_13|_14|     _15|     _16|     _17|     _18|     _19|     _20|     _21|     _22|     _23|     _24|     _25|     _26|     _27|     _28|     _29|     _30|     _31|     _32|     _33|     _34|     _35|_36|     _37|     _38|     _39|     _40|
+---+---+---+---+---+----+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---+--------+--------+--------+--------+
|  0|  1|  1|  5|  0|1382|  4| 15|  2|181|  1|  2|   |  2|68fd1e64|80e26c9b|fb936136|7b4723c4|25c83c98|7e0ccccf|de7995b8|1f89b562|a73ee5

In [8]:
# Define categorical and numerical columns
categoricalCols = []
numericalCols = []
numericalColsImputed = []
numericalColsLog = []
for c in range(2,41):
    col = "_"+str(c)
    colImp = str(c)+"_imp"
    colLog = str(c)+"_log"
    if (c < 15):
        numericalCols.append(col)
        numericalColsImputed.append(colImp)
        numericalColsLog.append(colLog)
    else:
        categoricalCols.append(col)

In [301]:
df.select(numericalCols).show(10)

+---+---+---+---+-----+---+---+---+---+---+---+---+---+
| _2| _3| _4| _5|   _6| _7| _8| _9|_10|_11|_12|_13|_14|
+---+---+---+---+-----+---+---+---+---+---+---+---+---+
|  1|  1|  5|  0| 1382|  4| 15|  2|181|  1|  2|   |  2|
|  2|  0| 44|  1|  102|  8|  2|  2|  4|  1|  1|   |  4|
|  2|  0|  1| 14|  767| 89|  4|  2|245|  1|  3|  3| 45|
|   |893|   |   | 4392|   |  0|  0|  0|   |  0|   |   |
|  3| -1|   |  0|    2|  0|  3|  0|  0|  1|  1|   |  0|
|   | -1|   |   |12824|   |  0|  0|  6|   |  0|   |   |
|   |  1|  2|   | 3168|   |  0|  1|  2|   |  0|   |   |
|  1|  4|  2|  0|    0|  0|  1|  0|  0|  1|  1|   |  0|
|   | 44|  4|  8|19010|249| 28| 31|141|   |  1|   |  8|
|   | 35|   |  1|33737| 21|  1|  2|  3|   |  1|   |  1|
+---+---+---+---+-----+---+---+---+---+---+---+---+---+
only showing top 10 rows



In [9]:
stages = [] # stages in our Pipeline

In [12]:
from pyspark.sql.functions import when
for col in numericalCols:
    df = df.withColumn(col, df[col].cast("double"))
    df= df.withColumn(col, when(df[col]<0, None).otherwise(df[col]))
#Also do this for column 0
df = df.withColumn("_1", df["_1"].cast("double"))

In [304]:
#Confirming that numerical columns with -1s etc are changed to Null
df.select(numericalCols).show(10)

+----+-----+----+----+-------+-----+----+----+-----+----+---+----+----+
|  _2|   _3|  _4|  _5|     _6|   _7|  _8|  _9|  _10| _11|_12| _13| _14|
+----+-----+----+----+-------+-----+----+----+-----+----+---+----+----+
| 1.0|  1.0| 5.0| 0.0| 1382.0|  4.0|15.0| 2.0|181.0| 1.0|2.0|null| 2.0|
| 2.0|  0.0|44.0| 1.0|  102.0|  8.0| 2.0| 2.0|  4.0| 1.0|1.0|null| 4.0|
| 2.0|  0.0| 1.0|14.0|  767.0| 89.0| 4.0| 2.0|245.0| 1.0|3.0| 3.0|45.0|
|null|893.0|null|null| 4392.0| null| 0.0| 0.0|  0.0|null|0.0|null|null|
| 3.0| null|null| 0.0|    2.0|  0.0| 3.0| 0.0|  0.0| 1.0|1.0|null| 0.0|
|null| null|null|null|12824.0| null| 0.0| 0.0|  6.0|null|0.0|null|null|
|null|  1.0| 2.0|null| 3168.0| null| 0.0| 1.0|  2.0|null|0.0|null|null|
| 1.0|  4.0| 2.0| 0.0|    0.0|  0.0| 1.0| 0.0|  0.0| 1.0|1.0|null| 0.0|
|null| 44.0| 4.0| 8.0|19010.0|249.0|28.0|31.0|141.0|null|1.0|null| 8.0|
|null| 35.0|null| 1.0|33737.0| 21.0| 1.0| 2.0|  3.0|null|1.0|null| 1.0|
+----+-----+----+----+-------+-----+----+----+-----+----+---+---

In [11]:
#Pipeline Stages- 6 of them
#Stage 1 - First transform numerical columns with imputer
from pyspark.ml.feature import Imputer
imputer = Imputer(inputCols=numericalCols, outputCols=numericalColsImputed)
stages += [imputer]
#imputer_model = imputer.fit(df)

#Stage 2 - Compute log transforms
from pyspark.ml.feature import SQLTransformer
sqlTrans = SQLTransformer(statement="SELECT *, \
                          log(2_imp+1) AS 2_log, \
                          log(3_imp+1) AS 3_log, \
                          log(4_imp+1) AS 4_log, \
                          log(5_imp+1) AS 5_log, \
                          log(6_imp+1) AS 6_log, \
                          log(7_imp+1) AS 7_log, \
                          log(8_imp+1) AS 8_log, \
                          log(9_imp+1) AS 9_log, \
                          log(10_imp+1) AS 10_log, \
                          log(11_imp+1) AS 11_log, \
                          log(12_imp+1) AS 12_log, \
                          log(13_imp+1) AS 13_log, \
                          log(14_imp+1) AS 14_log \
                          FROM __THIS__")
stages += [sqlTrans]

#Stage 3 - Assemble the column vectors
from pyspark.ml.feature import VectorAssembler
numericalAssembler = VectorAssembler(inputCols=numericalColsLog, outputCol="log_numerical_feature_vec")
stages += [numericalAssembler]

#Stage 4 - Now normalize these numerical features
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="log_numerical_feature_vec", outputCol="scaled_features")
stages += [scaler]

#Stage 5 - Next transform Categorical Features with FeatureHasher
from pyspark.ml.feature import FeatureHasher
hasher = FeatureHasher(numFeatures=256, inputCols=categoricalCols,outputCol="categorical_features")
stages += [hasher]

#Stage 6 - Now create vector with numerical and categorical features
finalAssembler = VectorAssembler(inputCols=["scaled_features", "categorical_features"], outputCol="features")
stages += [finalAssembler]

In [306]:
#Dataframe before pipeline
df.show(2)

+---+---+---+----+---+------+---+----+---+-----+---+---+----+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---+--------+--------+--------+--------+
| _1| _2| _3|  _4| _5|    _6| _7|  _8| _9|  _10|_11|_12| _13|_14|     _15|     _16|     _17|     _18|     _19|     _20|     _21|     _22|     _23|     _24|     _25|     _26|     _27|     _28|     _29|     _30|     _31|     _32|     _33|     _34|     _35|_36|     _37|     _38|     _39|     _40|
+---+---+---+----+---+------+---+----+---+-----+---+---+----+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---+--------+--------+--------+--------+
|0.0|1.0|1.0| 5.0|0.0|1382.0|4.0|15.0|2.0|181.0|1.0|2.0|null|2.0|68fd1e64|80e26c9b|fb936136|7b4723c4|25c83c98|7e0cc

In [307]:
#Finally create a pipeline and verify
from pyspark.ml import Pipeline
partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(df)
preppedDataDF = pipelineModel.transform(df)

In [308]:
preppedDataDF.select('_1','features').show(2, False)

+---+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_1 |features                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
+---+-----------------------------------------------------------------------------

In [309]:
# Keep relevant columns
selectedcols = ["_1", "features"]
dataset = preppedDataDF.select(selectedcols)
display(dataset)

DataFrame[_1: double, features: vector]

In [310]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)
print(trainingData.count())
print(testData.count())

70022
29978


In [311]:
#Run logistic regression
from pyspark.ml.classification import LogisticRegression


# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="_1", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [312]:
predictions = lrModel.transform(testData)

In [313]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="_1")
evaluator.evaluate(predictions)

0.7385414763574385

### 4.4.1 Logistic Regression with 100% dataset

In [5]:
# read in  training data and convert to dataframe
start = time.time()
train_sample = sc.textFile('data/train.txt')\
                 .map(lambda x: x.split('\t'))\
                 .toDF().repartition(16).write.parquet("train")
print(f"\n... Completed in {time.time() - start} seconds")

In [6]:
#Read the full dataset into a dataframe
start = time.time()
df = spark.read.parquet("train")

df.show(2)
print(f"\n... Completed in {time.time() - start} seconds")

+---+---+---+---+---+-----+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---+--------+--------+--------+--------+
| _1| _2| _3| _4| _5|   _6| _7| _8| _9|_10|_11|_12|_13|_14|     _15|     _16|     _17|     _18|     _19|     _20|     _21|     _22|     _23|     _24|     _25|     _26|     _27|     _28|     _29|     _30|     _31|     _32|     _33|     _34|     _35|_36|     _37|     _38|     _39|     _40|
+---+---+---+---+---+-----+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---+--------+--------+--------+--------+
|  0|   |  3|  3|  3|11826|   |  0|  5|614|   |  0|   |  3|ae82ea21|f8c8e8f8|8117bcef|f18a04d8|4cf72387|fbad5c96|bcb6cb0a|1f89b562|a7

In [ ]:
from pyspark.sql.functions import when
for col in numericalCols:
    df = df.withColumn(col, df[col].cast("double"))
    df= df.withColumn(col, when(df[col]<0, None).otherwise(df[col]))
#Also do this for column 0
df = df.withColumn("_1", df["_1"].cast("double"))

In [13]:
#Finally create a pipeline and verify
start = time.time()

from pyspark.ml import Pipeline
partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(df)
preppedDataDF = pipelineModel.transform(df)

print(f"\n... Completed in {time.time() - start} seconds")


... Completed in 1984.4682896137238 seconds


In [14]:
# Keep relevant columns
start = time.time()
selectedcols = ["_1", "features"]
dataset = preppedDataDF.select(selectedcols)
display(dataset)
print(f"\n... Completed in {time.time() - start} seconds")

DataFrame[_1: double, features: vector]


... Completed in 0.06742119789123535 seconds


In [15]:
### Randomly split data into training and test sets. set seed for reproducibility
start = time.time()
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)
print(f"\n... Completed in {time.time() - start} seconds")


... Completed in 0.02073836326599121 seconds


In [16]:
# Create initial LogisticRegression model
#Run logistic regression
from pyspark.ml.classification import LogisticRegression

start = time.time()
lr = LogisticRegression(labelCol="_1", featuresCol="features", maxIter=20)

# Train model with Training Data
lrModel = lr.fit(trainingData)
print(f"\n... Completed in {time.time() - start} seconds")


... Completed in 3428.8369302749634 seconds


In [17]:
start = time.time()
predictions = lrModel.transform(testData)
print(f"\n... Completed in {time.time() - start} seconds")


... Completed in 0.05697965621948242 seconds


In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
start = time.time()
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="_1")
print(evaluator.evaluate(predictions))
print(f"\n... Completed in {time.time() - start} seconds")

0.7250146213638494

... Completed in 2776.656357526779 seconds


In [ ]:
print("The area under ROC for train set is {}".format(evaluator.evaluate(predictions)))


The area under ROC for train set is 0.7250146213638572


In [ ]:
start = time.time()
predictions = lrModel.transform(datasetTest)
print(f"\n... Completed in {time.time() - start} seconds")

In [ ]:
# Evaluate model
start = time.time()
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="_1")
print(evaluator.evaluate(predictions))
print(f"\n... Completed in {time.time() - start} seconds")

In [ ]:
print("The area under ROC for train set is {}".format(evaluator.evaluate(predict_train)))
print("The area under ROC for test set is {}".format(evaluator.evaluate(predict_test)))

In [ ]:
start = time.time()
# View best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability")
display(selected)
print(f"\n... Completed in {time.time() - start} seconds")